In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import plotly.graph_objects as go

In [2]:
# ✅ Load data using relative paths
df_process = pd.read_csv("../ProcessData_ClusterByZ_DBSCAN/output/ProcessData_ClusterByZ_DBSCAN.csv")
df_geometry = pd.read_csv("../5MethodGeometryBased/output/5MethodGeometryBased.csv")


In [3]:
# === 1. Load data ===

df_process['layer_id'] = df_process['layer_id'].astype(int)
df_geometry['layer_id'] = df_geometry['layer_id_5MethodGeometryBased'].astype(int)

In [4]:
# === 2. Align X by layer center ===
df_proc_bounds = df_process.groupby('layer_id')['x'].agg(min_x_proc='min', max_x_proc='max')
df_geom_bounds = df_geometry.groupby('layer_id')['X'].agg(min_x_geom='min', max_x_geom='max')
df_align = pd.merge(df_proc_bounds, df_geom_bounds, left_index=True, right_index=True)
df_align['center_proc'] = (df_align['min_x_proc'] + df_align['max_x_proc']) / 2
df_align['center_geom'] = (df_align['min_x_geom'] + df_align['max_x_geom']) / 2
df_align['shift'] = df_align['center_geom'] - df_align['center_proc']
shift_map = df_align['shift'].to_dict()

df_process['x_aligned'] = df_process.apply(
    lambda row: row['x'] + shift_map.get(row['layer_id'], 0),
    axis=1
)

# === 3. Interpolate process parameters ===
skipped_layers = []

def transfer_process_to_geometry_interpolated(layer_id, df_process, df_geometry):
    proc = df_process[df_process['layer_id'] == layer_id].sort_values('x_aligned')
    geom = df_geometry[df_geometry['layer_id_5MethodGeometryBased'] == layer_id].sort_values('X')

    if len(proc) < 2 or len(geom) < 2 or proc['x_aligned'].nunique() < 2:
        skipped_layers.append(layer_id)
        print(f"[!] Skipped layer {layer_id} — not enough unique points")
        return pd.DataFrame()

    interpolated = geom.copy()
    for col in ['I', 'U', 'WFS', 'speed']:
        interp_func = interp1d(proc['x_aligned'], proc[col], bounds_error=False, fill_value="extrapolate")
        interpolated[f'proc_{col}'] = interp_func(geom['X'].values)

    interpolated['matched_layer_id'] = layer_id
    return interpolated

common_layers = sorted(set(df_process['layer_id']) & set(df_geometry['layer_id_5MethodGeometryBased']))
geom_interp_all = pd.concat([
    transfer_process_to_geometry_interpolated(lid, df_process, df_geometry)
    for lid in common_layers
], ignore_index=True)

print("✅ Interpolation complete. Row count:", len(geom_interp_all))
if skipped_layers:
    print("⚠️ Skipped layers:", skipped_layers)

# === 4. 3D visualization ===
fig = go.Figure()

for lid in common_layers:
    if lid in skipped_layers:
        continue

    layer_data = geom_interp_all[geom_interp_all['matched_layer_id'] == lid]
    fig.add_trace(go.Scatter3d(
        x=layer_data['X'],
        y=layer_data['Y'],
        z=layer_data['Z'],
        mode='markers',
        marker=dict(size=2),
        name=f"Layer {lid}",
        hovertemplate=(
            "Layer " + str(lid) + "<br>"
            "X: %{x:.2f} mm<br>"
            "Y: %{y:.2f} mm<br>"
            "Z: %{z:.2f} mm<br>"
            "I: %{customdata[0]:.2f} A<br>"
            "U: %{customdata[1]:.2f} V<br>"
            "WFS: %{customdata[2]:.2f} m/min<br>"
            "Speed: %{customdata[3]:.2f} mm/s<br><extra></extra>"
        ),
        customdata=layer_data[['proc_I', 'proc_U', 'proc_WFS', 'proc_speed']].round(2).values
    ))

# === 5. Axis scale locking ===
x_range = [geom_interp_all['X'].min(), geom_interp_all['X'].max()]
y_range = [geom_interp_all['Y'].min(), geom_interp_all['Y'].max()]
z_range = [geom_interp_all['Z'].min(), geom_interp_all['Z'].max()]
axis_length = max(
    x_range[1] - x_range[0],
    y_range[1] - y_range[0],
    z_range[1] - z_range[0]
)

fig.update_layout(
    title="3D Geometry with Interpolated Process Parameters (All Layers)",
    scene=dict(
        xaxis=dict(title='X', range=[x_range[0], x_range[0] + axis_length]),
        yaxis=dict(title='Y', range=[y_range[0], y_range[0] + axis_length]),
        zaxis=dict(title='Z', range=[z_range[0], z_range[0] + axis_length])
    ),
    margin=dict(l=0, r=0, b=0, t=40),
    legend_title="Layer"
)

# fig.show()
pass


c:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\.venv\Lib\site-packages\scipy\interpolate\_interpolate.py:479: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\.venv\Lib\site-packages\scipy\interpolate\_interpolate.py:479: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\.venv\Lib\site-packages\scipy\interpolate\_interpolate.py:479: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\.venv\Lib\site-packages\scipy\interpolate\_interpolate.py:479: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo

✅ Interpolation complete. Row count: 22965


In [5]:
import os

# 📁 Path to the output directory next to this notebook
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

# 📄 Path to the HTML file
output_path = os.path.join(output_dir, "3D_GeometryInterpolatedProcessParameters_layers.html")

# 💾 Save the figure
fig.write_html(output_path)
print(f"✅ Plot saved: {output_path}")


✅ Plot saved: ./output\3D_GeometryInterpolatedProcessParameters_layers.html


In [6]:
# === 6. Save the result to CSV ===
from pathlib import Path

# Create the output directory (if it doesn't already exist)
output_dir = Path("output")
output_dir.mkdir(parents=True, exist_ok=True)

# Path to the output file
output_path = output_dir / "Interpolated_Geometry_With_Process_Params.csv"

# Save the DataFrame
geom_interp_all.to_csv(output_path, index=False)

print(f"✅ Saved to: {output_path.resolve()}")


✅ Saved to: C:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\src\ProcessDataIntegration\output\Interpolated_Geometry_With_Process_Params.csv
